# BentoML Example: Sentiment Analysis with Scikit-learn


[BentoML](http://bentoml.ai) is an open source framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

This notebook demonstrates how to use BentoML to turn a scikit-learn model into a docker image containing a REST API server serving this model, how to use your ML service built with BentoML as a CLI tool, and how to distribute it a pypi package.


*The example is based on [this notebook](https://github.com/crawles/sentiment_analysis_twitter_model/blob/master/build-sentiment-classifier.ipynb), using dataset from [Sentiment140](http://help.sentiment140.com/for-students/)*

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=nb&ea=open&el=official-example&dt=sklearn-sentiment-clf)

In [ ]:
!pip install bentoml
!pip install sklearn pandas numpy

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

import bentoml

# Prepare Dataset

In [ ]:
%%bash

if [ ! -f ./trainingandtestdata.zip ]; then
    wget -q http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    unzip -n trainingandtestdata.zip
fi

In [ ]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns

# Model Training

In [ ]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df = 100,
                                                        ngram_range = (1,1),
                                                        stop_words = 'english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(dftrain.text, dftrain.polarity)

In [ ]:
Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]
print(classification_report(ytest,sentiment_lr.predict(Xtest)))

In [ ]:
sentiment_lr.predict([Xtest[0]])

# Define ML Service with BentoML

In [ ]:
%%writefile sentiment_lr_model.py
import pandas as pd
import bentoml
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([PickleArtifact('sentiment_lr')])
@bentoml.env(pip_dependencies=["scikit-learn", "pandas"])
class SentimentLRModel(bentoml.BentoService):

    @bentoml.api(DataframeHandler, typ='series')
    def predict(self, series):
        """
        predict expects pandas.Series as input
        """        
        return self.artifacts.sentiment_lr.predict(series)

# Save BentoML service archive

In [ ]:
from sentiment_lr_model import SentimentLRModel

# Initialize bentoML model with artifacts

bento_model = SentimentLRModel.pack(
    sentiment_lr=sentiment_lr
)

# Save bentoML model to directory
saved_path = bento_model.save("/tmp/bento")

# Load BentoML Service from archive

In [ ]:
import bentoml

# Load exported bentoML model archive from path
bento_model = bentoml.load(saved_path)

# Call predict on the restored sklearn model
bento_model.predict(pd.Series(["hello", "hi"]))

# "pip install" a BentoML archive

BentoML user can directly pip install saved BentoML archive with `pip install {saved_path}`,  and use it as a regular python package.

In [ ]:
!pip install {saved_path}

In [ ]:
# Your bentoML model class name will become packaged name
import SentimentLRModel

ms = SentimentLRModel.load() # call load to ensure all artifacts are loaded
ms.predict(pd.Series(["stupid", "awesome"]))

# CLI access

`pip install saved_path` also installs a CLI tool for accessing the BentoML service

In [ ]:
!SentimentLRModel info

In [ ]:
!SentimentLRModel --help

In [ ]:
!SentimentLRModel predict --help

In [ ]:
# Run prediction with sample input
!SentimentLRModel predict --input='["some new text, sweet noodles", "happy time", "sad day"]'

In [ ]:
# OpenAPI docs for generating API Client
!SentimentLRModel docs

# Run REST API server locally

In [ ]:
!bentoml serve {saved_path}

### Send prediction request to REST API server

*Run the following command in terminal to make a HTTP request to the API server*
```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '["some new text, sweet noodles", "happy time", "sad day"]' \
localhost:5000/predict
```

You can also view all availabl API endpoints at [localhost:5000](localhost:5000), or look at prometheus metrics at [localhost:5000/metrics](localhost:5000/metrics) in browser.

# Run REST API server with Docker

** _Note: `docker` is not available when running in Google Colaboratory_

### 1) build docker image with saved Bento and tag it (e.g. sentiment-lr-model)

In [ ]:
!cd {saved_path} && docker build -t sentiment-lr-model .

### 2) run docker image and expose port 5000

In [ ]:
!docker run -p 5000:5000 sentiment-lr-model

### 3) Similarly use the following command to query the REST server in Docker

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '["some new text, sweet noodles", "happy time", "sad day"]' \
localhost:5000/predict
```